In [110]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np

In [111]:
analyser = SentimentIntensityAnalyzer()

In [112]:
news_dir = 'data/bucket/news/'
twitter_dir = 'data/bucket/twitter/'
influencers = ['BarackObama','cnnbrk', 'KimKardashian', 'Trump']
sources = ['New York Times', 'CNN', 'Washington Post','NYT_ALL']

In [113]:
def sentiment_analyzer_scores(sentence):
    try:
        score = analyser.polarity_scores(sentence)
        return score['compound']
    except:
        print('Failed to score content:',sentence)
        return 0
    
#     print("{:-<40} {}".format(sentence, str(score)))

In [114]:
#TODO ADD CUSTOM HEURISTICS
def heuristic_1(df):
    senti = sum(df['sentiment_score']*df['number_of_likes'].apply(np.log))
    return senti
    

In [115]:
def sentimentTwiiter(influencers):
    for influencer in influencers:
        print('Processing:', influencer)
        df = pd.read_csv(twitter_dir+influencer+'.csv')
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Date'] = pd.to_datetime(df['Date'].dt.normalize())
        df['sentiment_score'] = df['content'].apply(sentiment_analyzer_scores)
        df['number_of_likes'] = df['number_of_likes'].map(lambda x: x if isinstance(x, int) else 0)
        df2 = df.groupby('Date').sum()
        df2 = df2.astype({'number_of_likes': 'int64'})
        df2['heuristic_score'] = df.groupby('Date').apply(heuristic_1)
        df2 = df2.reset_index()
        df2 = df2[['Date','sentiment_score', 'heuristic_score']]
        df2.to_csv(twitter_dir+influencer+'_sentiment.csv', index=False)
    print('done')

In [116]:
def sentimentNews(sources):
    for source in sources:
        print('Processing:', source)
        df = pd.read_csv(news_dir+source+'.csv')
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Date'] = pd.to_datetime(df['Date'].dt.normalize())
        df['sentiment_score'] = df['title'].apply(sentiment_analyzer_scores)
        df2 = df.groupby('Date').sum()
        df2 = df2.reset_index()
        df2 = df2[['Date','sentiment_score']]
        df2.to_csv(news_dir+source+'_sentiment.csv', index=False)
    print('done')

In [118]:
#AUTOMATION
sentimentTwiiter(influencers)
sentimentNews(sources)

Processing: BarackObama
Processing: cnnbrk
Processing: KimKardashian
Processing: Trump
done
Processing: New York Times
Processing: CNN
Processing: Washington Post
Processing: NYT_ALL
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Failed to score content: nan
Faile

In [79]:
df = pd.read_csv(twitter_dir+'BarackObama.csv', parse_dates=['Date'])
df['Date'] = pd.to_datetime(df['Date'].dt.normalize())

In [80]:
df2 = df.groupby('Date').sum()
df2['heuristic_score'] = df.groupby('Date').apply(heuristic_1)

In [90]:
df2.columns

Index(['number_of_likes', 'number_of_shares', 'sentiment_score',
       'heuristic_score'],
      dtype='object')

In [91]:
df2 = df2.reset_index()

In [92]:
df2[['Date','sentiment_score', 'heuristic_score']]

,Date,sentiment_score,heuristic_score
0,2014-01-08,2.1428,14.570481
1,2014-01-09,0.5106,4.453833
2,2014-01-10,0.4404,3.208839
3,2014-01-11,0.7096,5.263515
4,2014-01-12,-0.0949,-0.853277
...,...,...,...
691,2016-12-07,-0.7224,-5.853238
692,2016-12-08,1.5873,14.700298
693,2016-12-09,-0.3821,-3.302238
694,2016-12-10,0.6381,4.420319


In [65]:
df2.groupby('Date').aapl

In [66]:
df3 = df2.groupby('Date').apply(heuristic_1)

In [67]:
df3

Date
2014-01-08    14.570481
2014-01-09     4.453833
2014-01-10     3.208839
2014-01-11     5.263515
2014-01-12    -0.853277
                ...    
2016-12-07    -5.853238
2016-12-08    14.700298
2016-12-09    -3.302238
2016-12-10     4.420319
2017-10-01     4.454555
Length: 696, dtype: float64

In [27]:
df['Score'] = df['title'].apply(sentiment_analyzer_scores)

In [28]:
df

,title,author,Date,Score
0,37 Who Saw Murder Didn't Call the Police - The...,NaN,2014-05-08,-0.6908
1,The Pillow Cure - The New York Times,Penelope Green,2015-03-26,0.0000
2,John Ashbery: By the Book - The New York Times,NaN,2015-05-13,0.0000
3,"Drink to Your Health (in Moderation), the Scie...",Aaron E. Carroll,2015-12-22,0.0000
4,Simple Rules for Healthy Eating - The New York...,Aaron E. Carroll,2015-12-28,0.4019
...,...,...,...,...
30252,How a Trump Adviser Repeatedly Sought a Meetin...,Troy Griggs,NaT,0.0000
30253,"Satellite Images Show 1,800 Buildings Destroye...",Anjali Singhvi,NaT,-0.6808
30254,Trump Could Save More Than $1 Billion Under Hi...,Jesse Drucker,NaT,0.4939
30255,Opinion | The Republican’s Guide to Presidenti...,The Editorial Board,NaT,0.0000


In [30]:
df.to_csv('New York Times.csv', index=False)